In [5]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [64]:
import pandas as pd
import numpy as np
import importlib.metadata
import mysql.connector

from datetime import datetime

In [67]:
# Función para insertar un coche en la base de datos
def insertar_coche(df_coches):
    # Conexión a la base de datos de MySQL
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Whereisbotus1996",
        database="coches_segunda_mano"
    )
    
    # Definir las columnas a insertar
    columnas_coches = [
        "marca", "modelo", "localizacion", "tipo_combustible", "kilometros", 
        "tipo_carroceria", "tipo_cambio", "potencia_cv", "garantia", 
        "distintivo_ambiental", "precio", "peso_masa_kg", "puertas", 
        "plazas", "cilindros", "co2", "deposito", "month", "year", "fecha_extraccion"
    ]

    cursor = conn.cursor()
    for index, row in df_coches.iterrows():
        coche_data = {
            'marca': row['marca'],
            'modelo': row['modelo'],
            'localizacion': row['localizacion'],
            'tipo_combustible': row['tipo_combustible'],
            'kilometros': row['kilometros'],
            'tipo_carroceria': row['tipo_carroceria'],
            'tipo_cambio': row['tipo_combustible'],
            'potencia_cv': row['potencia_cv'],
            'garantia': row['garantia'],
            'distintivo_ambiental': row['distintivo_ambiental'],
            'precio': row['precio'],
            'peso_masa_kg': row['peso_masa_kg'],
            'puertas': row['puertas'],
            'plazas': row['plazas'],
            'cilindros': row['cilindros'],
            'co2': row['co2'],
            'deposito': row['deposito'],
            'month': row['month'],
            'year': row['year'],
            'fecha_extraccion': datetime.now()
        }

        valores = (
            coche_data['marca'],
            coche_data['modelo'],
            coche_data['localizacion'],
            coche_data['tipo_combustible'],
            coche_data['kilometros'],
            coche_data['tipo_carroceria'],
            coche_data['tipo_cambio'],
            coche_data['potencia_cv'],
            coche_data['garantia'],
            coche_data['distintivo_ambiental'],
            coche_data['precio'],
            coche_data['peso_masa_kg'],
            coche_data['puertas'],
            coche_data['plazas'],
            coche_data['cilindros'],
            coche_data['co2'],
            coche_data['deposito'],
            coche_data['month'],
            coche_data['year'],
            coche_data['fecha_extraccion']
        )

        query = f"""
            INSERT INTO coches_segunda_mano ({', '.join(columnas_coches)})
            VALUES ({', '.join(['%s'] * len(valores))});
        """
        
        cursor.execute(query, valores)

    conn.commit()
    cursor.close()
    conn.close()
    print("Datos insertados correctamente")
    

In [81]:
#Probamos la funcion

coche_prueba = {
    "marca": ["Toyota"],
    "modelo": ["Corolla"],
    "localizacion": ["Madrid"],
    "tipo_combustible": ["Gasolina"],
    "kilometros": [15000],
    "tipo_carroceria": ["Sedán"],
    "tipo_cambio": ["Manual"],
    "potencia_cv": [130],
    "garantia": ["1 año"],
    "distintivo_ambiental": ["C"],
    "precio": [15000],
    "peso_masa_kg": [1200],
    "puertas": [4],
    "plazas": [5],
    "cilindros": [4],
    "co2": [110],
    "deposito": [50],
    "month": [datetime.now().month],
    "year": [datetime.now().year],
    "fecha_extraccion": [datetime.now()]
}

df_coches_prueba = pd.DataFrame(coche_prueba)

# Llamada a la función para insertar los datos del coche de prueba
insertar_coche(df_coches_prueba)

ProgrammingError: 1054 (42S22): Unknown column 'localizacion' in 'field list'

In [32]:
#Cerrar la conexion al final del proceso
cursor.close()
conn.close()